# Exploration des données Online Retail

Ce notebook explore le jeu de données Online Retail disponible sur Kaggle pour la modélisation MMM.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
import os
import sys

# Configuration de Matplotlib
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

# Ajouter le répertoire parent au chemin Python
sys.path.append('..')

In [ ]:
# Initialiser une session Spark
spark = SparkSession.builder \
    .appName("online_retail_exploration") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

In [ ]:
# Charger les données
retail_path = "../data/online_retail.csv"
retail_df = spark.read.csv(retail_path, header=True, inferSchema=True)

# Afficher le schéma
print("Schéma des données:")
retail_df.printSchema()

# Aperçu des données
print("\nAperçu des données:")
retail_df.show(5)

In [ ]:
# Statistiques descriptives
print("Statistiques descriptives:")
retail_df.describe().show()

In [ ]:
# Convertir en pandas pour l'analyse exploratoire
pandas_df = retail_df.toPandas()
pandas_df['InvoiceDate'] = pd.to_datetime(pandas_df['InvoiceDate'])

# Examiner les valeurs nulles
print("Valeurs manquantes par colonne:")
print(pandas_df.isnull().sum())

In [ ]:
# Calculer le montant total par transaction
pandas_df['TotalAmount'] = pandas_df['Quantity'] * pandas_df['UnitPrice']

# Distribution du montant total
plt.figure(figsize=(10, 6))
sns.histplot(pandas_df['TotalAmount'].clip(0, 1000), bins=50)
plt.title('Distribution des montants de transaction (écrêté à 1000)')
plt.xlabel('Montant total (£)')
plt.ylabel('Fréquence')
plt.show()

In [ ]:
# Analyser les ventes quotidiennes
daily_sales = pandas_df.groupby(pandas_df['InvoiceDate'].dt.date).agg(
    revenue=('TotalAmount', 'sum'),
    transactions=('InvoiceNo', 'nunique'),
    unique_customers=('CustomerID', 'nunique')
).reset_index()

# Tracer les ventes quotidiennes
plt.figure(figsize=(14, 7))
plt.plot(daily_sales['InvoiceDate'], daily_sales['revenue'])
plt.title('Chiffre d\'affaires quotidien')
plt.xlabel('Date')
plt.ylabel('Chiffre d\'affaires (£)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyser les ventes par pays
country_sales = pandas_df.groupby('Country').agg(
    revenue=('TotalAmount', 'sum'),
    transactions=('InvoiceNo', 'nunique'),
    unique_customers=('CustomerID', 'nunique')
).reset_index().sort_values('revenue', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x='revenue', y='Country', data=country_sales.head(10))
plt.title('Chiffre d\'affaires par pays (top 10)')
plt.xlabel('Chiffre d\'affaires (£)')
plt.ylabel('Pays')
plt.show()

In [ ]:
# Analyser les patterns hebdomadaires
pandas_df['DayOfWeek'] = pandas_df['InvoiceDate'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

weekly_sales = pandas_df.groupby('DayOfWeek').agg(
    revenue=('TotalAmount', 'sum'),
    transactions=('InvoiceNo', 'nunique')
).reindex(day_order).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='DayOfWeek', y='revenue', data=weekly_sales)
plt.title('Chiffre d\'affaires par jour de la semaine')
plt.xlabel('Jour de la semaine')
plt.ylabel('Chiffre d\'affaires (£)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Analyser les patterns mensuels
pandas_df['Month'] = pandas_df['InvoiceDate'].dt.strftime('%Y-%m')
monthly_sales = pandas_df.groupby('Month').agg(
    revenue=('TotalAmount', 'sum'),
    transactions=('InvoiceNo', 'nunique'),
    unique_customers=('CustomerID', 'nunique')
).reset_index()

plt.figure(figsize=(14, 7))
sns.barplot(x='Month', y='revenue', data=monthly_sales)
plt.title('Chiffre d\'affaires mensuel')
plt.xlabel('Mois')
plt.ylabel('Chiffre d\'affaires (£)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Analyser les retours (quantities < 0)
returns_df = pandas_df[pandas_df['Quantity'] < 0]
print(f"Nombre de lignes avec des quantités négatives (retours): {len(returns_df)}")
print(f"Pourcentage des transactions qui sont des retours: {len(returns_df)/len(pandas_df)*100:.2f}%")

In [ ]:
# Top produits par chiffre d'affaires
product_sales = pandas_df.groupby('Description').agg(
    revenue=('TotalAmount', 'sum'),
    quantity=('Quantity', 'sum'),
    transactions=('InvoiceNo', 'nunique')
).reset_index().sort_values('revenue', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='revenue', y='Description', data=product_sales.head(15))
plt.title('Top 15 produits par chiffre d\'affaires')
plt.xlabel('Chiffre d\'affaires (£)')
plt.ylabel('Produit')
plt.tight_layout()
plt.show()

In [ ]:
# Décomposer les ventes par heure de la journée
pandas_df['Hour'] = pandas_df['InvoiceDate'].dt.hour
hourly_sales = pandas_df.groupby('Hour').agg(
    revenue=('TotalAmount', 'sum'),
    transactions=('InvoiceNo', 'nunique')
).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='Hour', y='revenue', data=hourly_sales)
plt.title('Chiffre d\'affaires par heure de la journée')
plt.xlabel('Heure')
plt.ylabel('Chiffre d\'affaires (£)')
plt.xticks(range(24))
plt.show()

In [ ]:
# Analyser la fréquence d'achat des clients
customer_frequency = pandas_df.groupby('CustomerID').agg(
    num_transactions=('InvoiceNo', 'nunique'),
    total_spend=('TotalAmount', 'sum'),
    first_purchase=('InvoiceDate', 'min'),
    last_purchase=('InvoiceDate', 'max')
).reset_index()

customer_frequency['days_between_purchases'] = (customer_frequency['last_purchase'] - customer_frequency['first_purchase']).dt.days

plt.figure(figsize=(12, 6))
sns.histplot(customer_frequency['num_transactions'].clip(0, 20), bins=20)
plt.title('Distribution du nombre de transactions par client')
plt.xlabel('Nombre de transactions')
plt.ylabel('Nombre de clients')
plt.show()

In [ ]:
# Identifier les clients les plus fidèles
top_customers = customer_frequency.sort_values('total_spend', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x='total_spend', y='CustomerID', data=top_customers)
plt.title('Top 10 des clients les plus fidèles par chiffre d\'affaires')
plt.xlabel('Chiffre d\'affaires total (£)')
plt.ylabel('ID du client')
plt.show()

## Conclusion

Dans ce notebook, nous avons exploré plusieurs aspects des données de l'Online Retail dataset, tels que les ventes quotidiennes, hebdomadaires, mensuelles, ainsi que l'analyse des retours, des clients les plus fidèles, des produits les plus populaires, et plus encore. Cette exploration a permis d'obtenir une vue d'ensemble sur les tendances de vente et les comportements des clients, qui pourront être utilisés pour des analyses plus approfondies et des modèles prédictifs.